# **RIC1-CPDM Descriptives**

Input: 
* CPDM behavioral file from Psychopy [sub-####_ses-cpdm_task.csv]

Ouput: 
* CRDM CSV file [sub-####_ses-cpdm.csv]

In [1]:
import os
import os.path, sys
import numpy as np
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings("ignore")

analysis = "1"
base_proj_dir = "Z:/data/RIC/" ## base project directory
data_dir = "Z:/data/RIC/sourcedata/RIC{0}/".format(analysis) ## directory containing data
save_proj_dir = os.path.join(base_proj_dir, "derivatives/RIC{0}/descriptives/cpdm".format(analysis)) ## output directory
if not os.path.exists(save_proj_dir): ## new Ss
    os.makedirs(save_proj_dir) ## make new Ss save directory

In [4]:
descriptives = {"SubID":[]}
dict_keys    = ["RespRate", "Acc", "HighConfProp", "LowConfProp", 
                "HighConfCt", "LowConfCt", "ConfAvg"]
tags = ["ALL", "LVLR", "LVHR", "HVLR", "HVHR"] ## unique per analysis
runs = ["all_runs", 
        "low_vol_low_risk", "low_vol_high_risk", 
        "high_vol_low_risk", "high_vol_high_risk"] ## unique per analysis

for tag in tags:
    for label in dict_keys:
        descriptives["{0} {1}".format(tag, label)] = [] ## create dict keys/empty lists

sub_files = sorted(glob(os.path.join(data_dir, "23_IDM_*.csv"))) ## grab all CRDM task csvs files in folder
for i, curr_file in enumerate(sub_files): ## go through each subject file
    sub_id = os.path.basename(curr_file)[7:11] ## grab first 5 indices of filename string
    if not sub_id in descriptives["SubID"]: ## check if already in list
        descriptives["SubID"].append(os.path.basename(curr_file)[7:11]) ## append current subject ID to subs list

for idx, sub in enumerate(descriptives["SubID"]): ## iterate through Ss ID list
    sub_file = sorted(glob(os.path.join(data_dir, "23_IDM_{0}.csv".format(sub)))) ## grab Ss task csv
    sub_cols = ["run", "acc", "conf", "money", "choice"] ## trial variables
    sub_df   = pd.DataFrame(columns = sub_cols) ## new subject-specific dataframe w/ preset columns
    raw_df   = pd.read_csv(sub_file[0]) ## open current data file
    df       = raw_df.loc[(raw_df["cpdm_trial_type"] == "task") & (raw_df["cpdm_trial_resp.keys"].notnull())] ## CPDM task response trials only
    sub_df["run"]    = df["cpdm_run_dimension"] ## don't *need* a new df, just makes things simpler
    sub_df["acc"]    = df["cpdm_acc"]
    sub_df["conf"]   = df["cpdm_conf"]
    sub_df["money"]  = df["cpdm_money"]
    choice_dict      = {'q':-2, 'a':-1, 'l':1, 'p':2} ## remap choice responses to match CASANDRE conceptualization
    sub_df["choice"] = df["cpdm_trial_resp.keys"].replace(choice_dict) ## replace old choices with remapped values
    for j, run_type in enumerate(runs):
        run_df = pd.DataFrame() ## empty run df
        if run_type == "all_runs":
            run_df = sub_df ## excludes nonresp trials
            descriptives["ALL RespRate"].append(np.round(len(run_df["choice"]) / 
                                                            len(raw_df.loc[(raw_df["cpdm_trial_type"] == "task")]), 3))
        else:
            run_df = sub_df.loc[(sub_df["run"] == run_type)]
            descriptives["{0} RespRate".format(tags[j])].append(np.round(len(run_df["choice"]) / 
                                                                        len(raw_df.loc[(raw_df["cpdm_trial_type"] == "task") 
                                                                                        & (raw_df["cpdm_run_dimension"] == run_type)]), 3))
        descriptives["{0} Acc".format(tags[j])].append(np.round(sum(run_df["acc"]) / len(run_df["acc"]), 3))
        descriptives["{0} HighConfProp".format(tags[j])].append(np.round(sum(run_df["conf"]) / len(run_df["conf"]), 3))
        descriptives["{0} LowConfProp".format(tags[j])].append(np.round(1 - (sum(run_df["conf"]) / len(run_df["conf"])), 3))
        descriptives["{0} HighConfCt".format(tags[j])].append(sum(run_df["conf"]))
        descriptives["{0} LowConfCt".format(tags[j])].append(len(run_df["conf"]) - sum(run_df["conf"]))
        descriptives["{0} ConfAvg".format(tags[j])].append(np.round(np.mean(run_df["conf"]), 3))
descriptives_df = pd.DataFrame(descriptives)
descriptives_df["SubID"] = descriptives_df["SubID"].apply('="{}"'.format) ## keeps zeros padding for SubID using string format
filename = os.path.join(save_proj_dir, "ric{0}-cpdm-descriptives.csv".fomat(analysis))
descriptives_df.to_csv(filename, index = False) ## save csv without dataframe row indexing